# The Server Runs the Testing Process on the Plain Testing Dataset

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import h5py
import tenseal as ts
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('dark_background')
from pathlib import Path

print(f'torch version: {torch.__version__}')
print(f'tenseal version: {ts.__version__}')

torch version: 1.8.1+cu102
tenseal version: 0.3.10


In [14]:
project_path = Path.cwd().parent
print(f'project_path: {project_path}')

project_path: /home/dk/Desktop/projects/HESplit


Needed paths

In [15]:
output_path = project_path / 'outputs/2022-08-17/ptbxl_8192_batch8_epoch10_lr0.001'
train_results = output_path / 'client/train_results.csv'
server_weights_path = output_path / 'server/trained_server.pth'
client_weights_path = output_path / 'client/trained_client.pth'

## Averange training time and communication per epoch

In [21]:
df = pd.read_csv(train_results)
df

,Unnamed: 0,train_losses,train_accs,train_times (s),train_comms (Mb)
0,0,1.428363,0.473037,47137.754390,1.315382e+06
1,1,1.340563,0.559921,49576.303444,1.315382e+06
2,2,1.318824,0.582083,51080.919545,1.315384e+06
3,3,1.311381,0.589921,50115.627517,1.315383e+06
4,4,1.304193,0.596253,50307.490460,1.315385e+06
5,5,1.300183,0.601650,50347.440193,1.315383e+06
6,6,1.294759,0.605491,49745.436481,1.315383e+06
7,7,1.292731,0.608865,50259.522743,1.315384e+06
8,8,1.289596,0.610993,48534.438388,1.315386e+06
9,9,1.290500,0.610474,46244.582279,1.315385e+06


In [22]:
avg_times = df['train_times (s)'].mean()
print(f'average training time per epoch: {avg_times:.2f} s')
avg_comms = df['train_comms (Mb)'].mean()
print(f'average training communication per epoch: {avg_comms:.2f} Mb')

average training time per epoch: 49334.95 s
average training communication per epoch: 1315383.61 Mb


## Dataset

In [16]:
class PTBXL(Dataset):
    """
    The class used by the client to 
    load the PTBXL dataset

    Args:
        Dataset ([type]): [description]
    """
    def __init__(self, train=True):
        if train:
            with h5py.File(project_path/'data/ptbxl_train.hdf5', 'r') as hdf:
                self.x = hdf['X_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(project_path/'data/ptbxl_test.hdf5', 'r') as hdf:
                self.x = hdf['X_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

batch_size = 4
test_dataset = PTBXL(train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Loading the trained model

In [17]:
client = torch.load(client_weights_path)
server = torch.load(server_weights_path)

class ECGModel(nn.Module):
    def __init__(self) -> None:
        super(ECGModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=12, 
                                out_channels=16, 
                                kernel_size=7, 
                                padding=3,
                                stride=1)  # 16 x 1000
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 16 x 500
        self.conv2 = nn.Conv1d(in_channels=16, 
                                out_channels=8, 
                                kernel_size=5, 
                                padding=2)  # 8 x 500
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 8 x 250
        
        self.linear = nn.Linear(in_features=8*250,
                                out_features=5)
        self.softmax = nn.Softmax(dim=1)

        self.load_weights()

    def load_weights(self):
        self.conv1.weight.data = client["conv1.weight"]
        self.conv1.bias.data = client["conv1.bias"]
        self.conv2.weight.data = client["conv2.weight"]
        self.conv2.bias.data = client["conv2.bias"]
        self.linear.weight.data = server["W"]
        self.linear.bias.data = server["b"]

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 8*250)
        x = self.linear(x)
        x = self.softmax(x)
        return x

model = ECGModel()

## The testing loop

In [18]:
def test(model):
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        test_loss = 0.0
        correct, total = 0, 0
        for _, batch in enumerate(test_loader):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.item()
            correct += torch.sum(y_hat.argmax(dim=1) == y).item()
            total += len(y)
    print(f"test_loss: {(test_loss/len(test_loader)):.4f}, "
          f"test_acc: {((correct/total)*100):.2f}")

test(model.to(device))

test_loss: 1.3109, test_acc: 58.95
